# Creating different DTM (Count-Vectorizer, TFIDF Vectorizer, Binary (?) ... )

In [99]:
#Loading in dataset
import pandas as pd
from pandas import ExcelWriter, ExcelFile

df = pd.read_csv("export_dataframe.csv")
df.head()

,Field,Abstract
0,in 1971 goodman and rose predicted the occurre...,Physics
1,we present pseudopotential coefficients for th...,Physics
2,nose has modified newtonian dynamics so as to ...,Physics
3,we provide the first evidence for a holographi...,Physics
4,a technique is demonstrated for polarization d...,Physics


# Word Count + IDF

In [100]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

corpus = df['Field'].tolist()
vectorizer = CountVectorizer(stop_words = 'english')
vectorizer

X = vectorizer.fit_transform(corpus)
word_counts = X.toarray()

In [106]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
transformer.fit(word_counts)

df_idf = pd.DataFrame(transformer.idf_, index = vectorizer.get_feature_names(), columns = ['idf_weights'])
df_idf.sort_values(by=['idf_weights'], ascending = True).head()

,idf_weights
author,1.017646
sports,2.632695
international,2.661231
space,2.709758
results,2.732105


# Construction of TFIDF DTM

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(corpus)

first = tfidf_vectorizer_vectors[0] #TFIDF for first vector

vectors_tfidf = pd.DataFrame(first.T.todense(), index = tfidf_vectorizer.get_feature_names(), columns = ['tfidf'])
vectors_tfidf.sort_values(by=['tfidf'], ascending=False).head()


,tfidf
charge,0.418944
photocurrent,0.366539
voltage,0.210413
carrier,0.210413
power,0.201752


In [115]:
dataframe.shape # TFIDF for first vector (first abstract)

(24051, 1)

In [117]:
dataframe.head()

,tfidf
00,0.0
000,0.0
0001,0.0
00014,0.0
0003,0.0
